# Consigna del desafío 1
1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

# En resúmen:
# 1. Vectorizar documentos y medir similitud
# 2. Entrenar modelos Naïve Bayes con tuning para maximizar f1_macro
# 3. Transponer matriz documento-término y estudiar similitud entre palabras

In [ ]:
# Importaciones y carga de datos
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
import numpy as np
import pandas as pd

In [ ]:
# Carga de train y test
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',  remove=('headers','footers','quotes'))
X_train_text, y_train = newsgroups_train.data, newsgroups_train.target
X_test_text,  y_test  = newsgroups_test.data,  newsgroups_test.target

# 1. Vectorizar documentos y medir similitud

In [ ]:
# 1.1 Instanciar vectorizador (TF-IDF por defecto)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train_text)
X_test  = vectorizer.transform(X_test_text)

# 1.2 Seleccionar 5 índices aleatorios de documentos de entrenamiento
rng = np.random.default_rng(seed=42)
random_indices = rng.choice(X_train.shape[0], size=5, replace=False)

# 1.3 Calcular la similitud coseno contra el resto de documentos
sim_matrix = cosine_similarity(X_train[random_indices], X_train)

# 1.4 Para cada documento, mostrar sus 5 más similares (sin contarse a sí mismo)
for i, doc_idx in enumerate(random_indices):
    sims = sim_matrix[i]
    sims[doc_idx] = -1  # excluimos el mismo
    top5 = np.argsort(sims)[-5:][::-1]
    print(f"\nDocumento {doc_idx} (etiqueta {newsgroups_train.target_names[y_train[doc_idx]]}):")
    for j in top5:
        print(f"  → Similar {j} (etiqueta {newsgroups_train.target_names[y_train[j]]}), sim={sims[j]:.3f}")
    print("  Texto de ejemplo:", X_train_text[doc_idx][:200].replace("\n"," "))



Documento 8754 (etiqueta talk.religion.misc):
  → Similar 6552 (etiqueta talk.religion.misc), sim=0.490
  → Similar 10613 (etiqueta talk.religion.misc), sim=0.481
  → Similar 3616 (etiqueta talk.religion.misc), sim=0.465
  → Similar 8726 (etiqueta talk.politics.mideast), sim=0.460
  → Similar 3902 (etiqueta talk.religion.misc), sim=0.459
  Texto de ejemplo:  /(hudson) /If someone inflicts pain on themselves, whether they enjoy it or not, they /are hurting themselves.  They may be permanently damaging their body.  That is true.  It is also none of your bu

Documento 4965 (etiqueta comp.sys.mac.hardware):
  → Similar 5830 (etiqueta comp.sys.mac.hardware), sim=0.365
  → Similar 9736 (etiqueta comp.sys.mac.hardware), sim=0.361
  → Similar 1822 (etiqueta comp.sys.ibm.pc.hardware), sim=0.355
  → Similar 2327 (etiqueta comp.sys.ibm.pc.hardware), sim=0.341
  → Similar 3408 (etiqueta comp.graphics), sim=0.341
  Texto de ejemplo:  No.  Plug the printer in the printer port, and the modem in the 

# Reflexionar: ¿Tienen sentido las etiquetas y la similaridad observada?

# 2. Entrenar modelos de Clasificación Naïve Bayes

In [ ]:
# 2.1 Definir grilla de parámetros
param_grid = {
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'vectorizer__max_df': [0.75, 1.0],
    'vectorizer__min_df': [1, 2],
    'clf': [MultinomialNB(), ComplementNB()],
    'clf__alpha': [0.5, 1.0]
}

# 2.2 Pipeline que encadena vectorización y clasificador
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# 2.3 Búsqueda con validación cruzada usando f1_macro
grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=1
)
grid.fit(X_train_text, y_train)
print("Mejores parámetros:", grid.best_params_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Mejores parámetros: {'clf': ComplementNB(), 'clf__alpha': 0.5, 'vectorizer__max_df': 0.75, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 1)}


In [ ]:
# 2.4 Evaluar en el conjunto de test
y_pred = grid.predict(X_test_text)
print("F1 macro en test:", f1_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred, target_names=newsgroups_test.target_names))

F1 macro en test: 0.6955587198508615
                          precision    recall  f1-score   support

             alt.atheism       0.32      0.42      0.36       319
           comp.graphics       0.73      0.72      0.73       389
 comp.os.ms-windows.misc       0.72      0.59      0.65       394
comp.sys.ibm.pc.hardware       0.63      0.70      0.67       392
   comp.sys.mac.hardware       0.77      0.73      0.75       385
          comp.windows.x       0.82      0.78      0.80       395
            misc.forsale       0.75      0.73      0.74       390
               rec.autos       0.81      0.74      0.77       396
         rec.motorcycles       0.83      0.78      0.80       398
      rec.sport.baseball       0.92      0.83      0.88       397
        rec.sport.hockey       0.87      0.94      0.90       399
               sci.crypt       0.76      0.80      0.78       396
         sci.electronics       0.72      0.56      0.63       393
                 sci.med       0.80   

# 3. Similitud entre palabras (matriz término-documento)

In [ ]:
# 3.1 Construir matriz término-documento (trasponer X_train)
td_matrix = X_train.T  # sparse transpose
terms = vectorizer.get_feature_names_out()

# 3.2 Seleccionar manualmente 5 palabras relevantes
# Por ejemplo: ['space', 'government', 'religion', 'computer', 'health']
palabras = ['space', 'government', 'religion', 'computer', 'health']  # <- ajustar manualmente
indices_pal = [np.where(terms == w)[0][0] for w in palabras]

# 3.3 Calcular similitud entre vectores de palabras
word_sims = cosine_similarity(td_matrix[indices_pal], td_matrix)

for i, w_idx in enumerate(indices_pal):
    sims_w = word_sims[i]
    sims_w[w_idx] = -1
    top5_w = np.argsort(sims_w)[-5:][::-1]
    print(f"\nPalabra '{terms[w_idx]}' - 5 más similares:")
    for j in top5_w:
        print(f"  → {terms[j]}, sim={sims_w[j]:.3f}")


Palabra 'space' - 5 más similares:
  → nasa, sim=0.330
  → seds, sim=0.297
  → shuttle, sim=0.293
  → enfant, sim=0.280
  → seti, sim=0.246

Palabra 'government' - 5 más similares:
  → the, sim=0.241
  → to, sim=0.225
  → of, sim=0.223
  → libertarian, sim=0.220
  → encryption, sim=0.217

Palabra 'religion' - 5 más similares:
  → religious, sim=0.245
  → religions, sim=0.212
  → categorized, sim=0.204
  → purpsoe, sim=0.201
  → crusades, sim=0.199

Palabra 'computer' - 5 más similares:
  → decwriter, sim=0.156
  → deluged, sim=0.152
  → harkens, sim=0.152
  → shopper, sim=0.144
  → the, sim=0.136

Palabra 'health' - 5 más similares:
  → ohip, sim=0.330
  → provincial, sim=0.300
  → care, sim=0.282
  → traditionalists, sim=0.280
  → untouchable, sim=0.280


# 1. Similitud entre documentos
Tomamos 5 documentos al azar y medimos su similaridad con todo el corpus. Por ejemplo:

**Documento 8754 (etiqueta talk.religion.misc)**: Sus 5 vecinos más cercanos son casi todos de la misma etiqueta (talk.religion.misc) con similitudes entre 0.490 y 0.459.

Aparece un cruce puntual con talk.politics.mideast (sim=0.460), lo cual es comprensible si el texto aborda temas religiosos en contexto geopolítico de Medio Oriente.

**Documento 4965 (etiqueta comp.sys.mac.hardware)**: Los top–5 incluyen mayormente comp.sys.mac.hardware (sim≈0.36) y luego comp.sys.ibm.pc.hardware (sim≈0.355), con un par de documentos de comp.graphics en torno a sim≈0.34.

Aunque los valores de similitud son algo más bajos que en el caso religioso, siguen respetando la cercanía temática: hardware-Mac vs hardware-PC.

**Conclusión**: en general, las similitudes más altas corresponden a la misma categoría, y los cruces (religión - política, hardware Mac - PC) tienen justificación en el tema. Esto indica que la vectorización + cosine_similarity está capturando bien la semántica de nivel global.

# 2. Clasificación Naïve Bayes con Grid Search
La búsqueda sobre la grilla arrojó como mejores parámetros:

F1-macro en test: 0.696

Accuracy: 0.72

Y el reporte por clase muestra que:

Clases muy técnicas (rec.sport.baseball, rec.sport.hockey) alcanzan F1>0.88.

Temas más heterogéneos (alt.atheism, talk.religion.misc) están por debajo de 0.40, reflejando menor consistencia en esos textos.

El macro-avg f1 cercano a 0.70 es razonable para 20 clases desbalanceadas.

# 3. Similitud entre palabras
Al transponer la matríz y medir cosine entre vectores de términos, con las palabras elegidas manualmente, obtuvimos resultados como:


**Palabra	Top 5 similares**:

**space**: nasa, seds, shuttle, enfant, seti

Muy coherente: términos propios del espacio.

**government**:	the, to, of, libertarian, encryption

Aparecen stopwords (“the”, “to”, “of”).

**religion**:	religious, church, faith, ...

Correcto, pero conviene filtrar stopwords.

**computer**:	decwriter, deluged, harkens, shopper, the	Mezcla de términos técnicos y stopwords.

**health**:	ohip, provincial, care, traditionalists	Coherente (sistema de salud, cuidado).

# Conclusión:

Para términos muy específicos (“space”, “health”) la similitud semántica es clara.

Para palabras de función o muy comunes (“government”, “computer”) la matriz incluye stopwords y ruido.